## 1 Downloading The Dataset

In [ ]:
#Downloading dataset to train on.

#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [117]:
with open('../dataset.txt','r',encoding='utf-8') as data:
    text = data.read()
    
print("Total number of characters in the dataset : ",len(text))

Total number of characters in the dataset :  1115394


In [118]:
print('First 100 characters : ', text[:100])

First 100 characters :  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## 2 Processing The Dataset 

### 2.1 Building Vocabulary 

In [119]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print('Number of unique characters in the dataset =',vocab_size)
print('\nWhich are following : \n',''.join(chars))

Number of unique characters in the dataset = 65

Which are following : 
 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### 2.2 Building Tokenizer  (Custom Encoder, Decoder)

In [120]:
#mapping individual charaters to integers 

encoder = { char:i for i,char in enumerate(chars)}
decoder = { i:char for i,char in enumerate(chars)}

encode = lambda string: [encoder[char] for char in string]
decode = lambda integers: [decoder[i] for i in integers]

In [121]:
zk = encode('Zain Khalid')
print('Encoded Zain Khalid \n= ',zk)
print('\nDecoded ',zk,'\n=', ''.join(decode(zk)))

Encoded Zain Khalid 
=  [38, 39, 47, 52, 1, 23, 46, 39, 50, 47, 42]

Decoded  [38, 39, 47, 52, 1, 23, 46, 39, 50, 47, 42] 
= Zain Khalid


### 2.3 Encoding The Dataset 

In [122]:
import tensorflow as tf

In [123]:
data = tf.convert_to_tensor(encode(text), dtype=tf.int64)
print(data.shape, data.dtype)

(1115394,) <dtype: 'int64'>


In [124]:
print(data[:100])

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)


### 2.4 Splitting the dataset (Train,Validate) 

In [125]:
limit = int(0.9 * len(data))

train_data = data[:limit]
val_data = data[limit:]

In [126]:
print(train_data[:100])

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)


### 2.5 Chunking Dataset in Blocks (x,y) (To Train Transformer) 

#### Concept 

In [127]:
block_size = 8
train_data[:block_size+1]

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([18, 47, 56, 57, 58,  1, 15, 47, 58], dtype=int64)>

In [128]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for token in range(block_size):
    context = x[:token+1]
    target = y[token]
    print('for input: ',context.numpy().tolist(),'  target is: ',target.numpy().tolist())

for input:  [18]   target is:  47
for input:  [18, 47]   target is:  56
for input:  [18, 47, 56]   target is:  57
for input:  [18, 47, 56, 57]   target is:  58
for input:  [18, 47, 56, 57, 58]   target is:  1
for input:  [18, 47, 56, 57, 58, 1]   target is:  15
for input:  [18, 47, 56, 57, 58, 1, 15]   target is:  47
for input:  [18, 47, 56, 57, 58, 1, 15, 47]   target is:  58


#### Implementation 

In [129]:
batch_size = 4 #Number of independent input sequences to process in parallel for GPU
block_size = 8 #Maximum context length to make predictions
n_embd = 32

def get_batch(split):
    #generate small batches of input x & target y
    data = train_data if split == 'train' else val_data
    randPos = tf.dtypes.cast(tf.random.uniform((batch_size,), minval=0, maxval=(len(data)-block_size)), dtype=tf.int32)
    #print(randPos) # random positions in the whole datasets to grab block size chunks
    xbatch = tf.stack([data[i:i+block_size] for i in randPos])
    ybatch = tf.stack([data[i+1:i+block_size+1] for i in randPos])
    
    return xbatch, ybatch

xbatch, ybatch = get_batch('train')

print('inputs:')
print(xbatch.shape)
print(xbatch)

print('targets:')
print(ybatch.shape)
print(ybatch)

inputs:
(4, 8)
tf.Tensor(
[[63  1 61 43 39 49 52 43]
 [43  6  1 51 63  1 50 53]
 [ 1 58 46 43 51  1 44 59]
 [50 50  1 58 46 43 43  1]], shape=(4, 8), dtype=int64)
targets:
(4, 8)
tf.Tensor(
[[ 1 61 43 39 49 52 43 57]
 [ 6  1 51 63  1 50 53 56]
 [58 46 43 51  1 44 59 56]
 [50  1 58 46 43 43  1 52]], shape=(4, 8), dtype=int64)


In [130]:
for row in range(batch_size):
    for token in range(block_size):
        context = xbatch[row, :token+1]
        target = ybatch[row, token]
        print('for input: ',context.numpy().tolist(),'  target is: ',target.numpy().tolist())

for input:  [63]   target is:  1
for input:  [63, 1]   target is:  61
for input:  [63, 1, 61]   target is:  43
for input:  [63, 1, 61, 43]   target is:  39
for input:  [63, 1, 61, 43, 39]   target is:  49
for input:  [63, 1, 61, 43, 39, 49]   target is:  52
for input:  [63, 1, 61, 43, 39, 49, 52]   target is:  43
for input:  [63, 1, 61, 43, 39, 49, 52, 43]   target is:  57
for input:  [43]   target is:  6
for input:  [43, 6]   target is:  1
for input:  [43, 6, 1]   target is:  51
for input:  [43, 6, 1, 51]   target is:  63
for input:  [43, 6, 1, 51, 63]   target is:  1
for input:  [43, 6, 1, 51, 63, 1]   target is:  50
for input:  [43, 6, 1, 51, 63, 1, 50]   target is:  53
for input:  [43, 6, 1, 51, 63, 1, 50, 53]   target is:  56
for input:  [1]   target is:  58
for input:  [1, 58]   target is:  46
for input:  [1, 58, 46]   target is:  43
for input:  [1, 58, 46, 43]   target is:  51
for input:  [1, 58, 46, 43, 51]   target is:  1
for input:  [1, 58, 46, 43, 51, 1]   target is:  44
for

## 3 Without Transformer  

## Maths for self-attention mechanism

In [149]:
#We want a method to preserve context for each word and it's former words in the time block

In [150]:
# example of words and their context

for row in range(batch_size-3):
    for token in range(block_size):
        context = xbatch[row, :token+1]
        target = ybatch[row, token]
        print('for input: ',context.numpy().tolist(),'  target is: ',target.numpy().tolist())

for input:  [63]   target is:  1
for input:  [63, 1]   target is:  61
for input:  [63, 1, 61]   target is:  43
for input:  [63, 1, 61, 43]   target is:  39
for input:  [63, 1, 61, 43, 39]   target is:  49
for input:  [63, 1, 61, 43, 39, 49]   target is:  52
for input:  [63, 1, 61, 43, 39, 49, 52]   target is:  43
for input:  [63, 1, 61, 43, 39, 49, 52, 43]   target is:  57


In [151]:
xbatch.shape

TensorShape([4, 8])

### 3.1 Aggregating for context (simplest method) 

#### Version 1 

In [152]:
B, T, C = 4, 8, 2  # batch, time, channels
x = tf.random.normal((B, T, C), dtype=tf.float64)
print(x.shape)


(4, 8, 2)


In [153]:
xbow = tf.zeros((B, T, C), dtype=tf.float64)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1, :]  # (t, C)
        mean_value = tf.reduce_mean(xprev, axis=0)
        xbow = tf.tensor_scatter_nd_add(xbow, indices=[[b, t]], updates=[mean_value])


In [154]:
x[0]
#original data

<tf.Tensor: shape=(8, 2), dtype=float64, numpy=
array([[ 0.95310103, -0.33585035],
       [-0.89537386, -0.10109913],
       [-1.08746957, -0.23392829],
       [ 0.55400938, -3.05065275],
       [-0.4705944 , -1.86886873],
       [ 2.04610976, -0.4255422 ],
       [-1.76064349,  0.43410932],
       [-1.65781871,  0.70082522]])>

In [155]:
#aggregated data
xbow
#each upcoming row is contains the aggregate of all previous 

<tf.Tensor: shape=(4, 8, 2), dtype=float64, numpy=
array([[[ 0.95310103, -0.33585035],
        [ 0.02886359, -0.21847474],
        [-0.34324746, -0.22362592],
        [-0.11893325, -0.93038263],
        [-0.18926548, -1.11807985],
        [ 0.18329706, -1.00265691],
        [-0.09440874, -0.79740459],
        [-0.28983498, -0.61012586]],

       [[ 0.24474915,  0.5304757 ],
        [-0.0215186 ,  0.89519942],
        [-0.29395758,  0.44847173],
        [ 0.17669387,  0.5498084 ],
        [ 0.18008692,  0.3448098 ],
        [-0.0328576 ,  0.4573222 ],
        [-0.09852322,  0.57093864],
        [ 0.09555441,  0.39454034]],

       [[-0.45030911, -0.85046188],
        [ 0.61036958, -0.90442377],
        [ 0.601554  , -1.25840561],
        [ 0.25857243, -0.86086016],
        [ 0.28360905, -0.79047013],
        [ 0.316442  , -0.62021193],
        [ 0.02984569, -0.50866335],
        [-0.3375226 , -0.46764786]],

       [[ 0.18904454, -1.09819457],
        [ 0.18068477,  0.22778135],
       

### 3.2 Efficient aggregating for context (simplest method) 

#### Concept (Same achived through matrix multiplication trick)

In [156]:
import numpy as np

# Create a lower triangular matrix
a = tf.linalg.band_part(tf.ones((3, 3), dtype=tf.float64), -1, 0)
a = a / tf.reduce_sum(a, axis=1, keepdims=True)

# Create a random matrix with integer values between 0 and 10
b = tf.constant(np.random.randint(0, 10, size=(3, 2)).astype(np.float64))

# Perform matrix multiplication
c = tf.matmul(a, b)

# Print the results
print('a=')
print(a.numpy())
print('--')
print('b=')
print(b.numpy())
print('--')
print('c=')
print(c.numpy())

a=
[[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333333 0.33333333 0.33333333]]
--
b=
[[7. 8.]
 [5. 0.]
 [2. 7.]]
--
c=
[[7.         8.        ]
 [6.         4.        ]
 [4.66666667 5.        ]]


#### Version 2 (Reproducing xbow via matrix mulitplication instead of loop multiplication)

In [157]:
weights = tf.linalg.band_part(tf.ones((T, T), dtype=tf.float64), -1, 0) 
weights = weights / tf.reduce_sum(weights, axis=1, keepdims=True)

# Round each value to 3 decimal points for better visibility
weights_numpy_rounded = np.round(weights.numpy().tolist(), decimals=3)
weights_numpy_rounded

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.5  , 0.5  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.333, 0.333, 0.333, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.25 , 0.25 , 0.25 , 0.25 , 0.   , 0.   , 0.   , 0.   ],
       [0.2  , 0.2  , 0.2  , 0.2  , 0.2  , 0.   , 0.   , 0.   ],
       [0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.   , 0.   ],
       [0.143, 0.143, 0.143, 0.143, 0.143, 0.143, 0.143, 0.   ],
       [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]])

In [158]:
xbow2 = tf.matmul(weights, x) # Here (B,T,T) x (B,T,C) = (B,T,C).   where B in (B,T,T) was automatically created by tf

In [159]:
print('xbow = ',xbow[0].numpy())
print('\nSame as\n')
print('xbow2 = ',xbow2[0].numpy())

xbow =  [[ 0.95310103 -0.33585035]
 [ 0.02886359 -0.21847474]
 [-0.34324746 -0.22362592]
 [-0.11893325 -0.93038263]
 [-0.18926548 -1.11807985]
 [ 0.18329706 -1.00265691]
 [-0.09440874 -0.79740459]
 [-0.28983498 -0.61012586]]

Same as

xbow2 =  [[ 0.95310103 -0.33585035]
 [ 0.02886359 -0.21847474]
 [-0.34324746 -0.22362592]
 [-0.11893325 -0.93038263]
 [-0.18926548 -1.11807985]
 [ 0.18329706 -1.00265691]
 [-0.09440874 -0.79740459]
 [-0.28983498 -0.61012586]]


#### Version 3 (Using softmax) 

In [160]:
# Create lower triangular matrix and initialize wei with zeros
tril = tf.linalg.band_part(tf.ones((T, T), dtype=tf.float64), -1, 0)
weights = tf.zeros((T, T), dtype=tf.float64)

# Mask the upper triangular part with -inf
weights = tf.where(tf.equal(tril, 0), float('-inf'), weights)

# Apply softmax along the last dimension
weights = tf.nn.softmax(weights, axis=-1)

# Perform matrix multiplication
xbow3 = tf.matmul(weights, x)

In [161]:
print('xbow = ',xbow[0].numpy())
print('\nSame as\n')
print('xbow2 = ',xbow3[0].numpy())

xbow =  [[ 0.95310103 -0.33585035]
 [ 0.02886359 -0.21847474]
 [-0.34324746 -0.22362592]
 [-0.11893325 -0.93038263]
 [-0.18926548 -1.11807985]
 [ 0.18329706 -1.00265691]
 [-0.09440874 -0.79740459]
 [-0.28983498 -0.61012586]]

Same as

xbow2 =  [[ 0.95310103 -0.33585035]
 [ 0.02886359 -0.21847474]
 [-0.34324746 -0.22362592]
 [-0.11893325 -0.93038263]
 [-0.18926548 -1.11807985]
 [ 0.18329706 -1.00265691]
 [-0.09440874 -0.79740459]
 [-0.28983498 -0.61012586]]


### 3.3 Self-attention mechanism context (better method)  

#### Version 4 (Using Key, Query, Value upon version 3)

In [165]:
tf.random.set_seed(1337)

B, T, C = 4, 8, 32  # batch, time, channels
x = tf.random.normal((B, T, C))
x.shape

TensorShape([4, 8, 32])

In [166]:

# Let's see a single Head perform self-attention
head_size = 16

key = tf.keras.layers.Dense(head_size, use_bias=False)
query = tf.keras.layers.Dense(head_size, use_bias=False)
value = tf.keras.layers.Dense(head_size, use_bias=False)

print('x.shape : ', x.shape)
k = key(x)   # (B, T, 32) => (B, T, 16)
q = query(x) # (B, T, 32) => (B, T, 16)
print('k.shape : ', k.shape)
print('q.shape : ', q.shape)

wei = tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])* head_size**-0.5)  # (B, T, T)  #also scaling after transpose, check below to see why

print('wei.shape : ',wei.shape)

tril = tf.linalg.band_part(tf.ones((T, T), dtype=tf.float64), -1, 0)
wei = tf.where(tf.equal(tril, 0), float('-inf'), wei)

# Apply softmax along the last dimension
wei = tf.nn.softmax(wei, axis=-1)

v = value(x)
out = tf.matmul(wei, v)

print(out.shape)

x.shape :  (4, 8, 32)
k.shape :  (4, 8, 16)
q.shape :  (4, 8, 16)
wei.shape :  (4, 8, 8)
(4, 8, 16)


In [167]:
wei[0].numpy()

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.55890644, 0.44109353, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.36212704, 0.3926684 , 0.24520455, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.17977196, 0.32847154, 0.08661044, 0.40514615, 0.        ,
        0.        , 0.        , 0.        ],
       [0.2580921 , 0.03877385, 0.26799572, 0.25024772, 0.18489058,
        0.        , 0.        , 0.        ],
       [0.15109922, 0.45920232, 0.09098168, 0.07991469, 0.03213087,
        0.1866712 , 0.        , 0.        ],
       [0.2580665 , 0.03653046, 0.13164522, 0.01842463, 0.08224345,
        0.45676315, 0.01632656, 0.        ],
       [0.03404133, 0.00912395, 0.03316646, 0.1516977 , 0.09125233,
        0.00497304, 0.0166554 , 0.65908974]], dtype=float32)

In [168]:
#without scaling 
tf.nn.softmax(tf.constant([0.1, -0.2, 0.3, -0.2, 0.5]), axis=-1).numpy()

array([0.1924978 , 0.14260589, 0.23511736, 0.14260589, 0.287173  ],
      dtype=float32)

In [169]:
#without scaling the softmax favours larger values big time as visible
tf.nn.softmax(tf.constant([0.1, -0.2, 0.3, -0.2, 0.5])*8, axis=-1).numpy() 

array([0.03260834, 0.00295816, 0.1615102 , 0.00295816, 0.79996514],
      dtype=float32)

# Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

## 4 Bigram Language Model (With Transformer Block) 

### 4.1 Transformer Head Block 

In [170]:

class Head(tf.keras.Model):

    def __init__(self, head_size):
        super(Head, self).__init__()
        
        self.key = tf.keras.layers.Dense(head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(head_size, use_bias=False)
        self.tril = tf.linalg.band_part(tf.ones((block_size, block_size), dtype=tf.float64), -1, 0)

    def call(self, x):
        B, T, C = x.shape
        
        k = key(x)   # (B, T, C) => (B, T, head_size)
        q = query(x) # (B, T, C) => (B, T, head_size)
        
        #Compute Attention Score (Affinities)
        wei = tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])* C**-0.5)  # (B,T,C) @ (B, C, T) => (B, T, T)
        
        wei = tf.where(tf.equal(self.tril[:T,:T], 0), float('-inf'), wei) #Here we cut of future context which makes this head a decoder block

        # Apply softmax along the last dimension
        wei = tf.nn.softmax(wei, axis=-1)
        #Perform Weighted Aggregation of the values
        v = value(x) # (B,T,C)
        out = tf.matmul(wei, v) #(B,T,T) @ (B,T,C) -> (B,T,C)

        return out



### 4.2 Bigram Language Model  

In [174]:

class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size, n_embd):
        super(BigramLanguageModel, self).__init__()
        
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embd)
        self.position_embedding_table = tf.keras.layers.Embedding(block_size, n_embd)
        self.sa_head = tf.keras.layers.Dense(n_embd)
        self.lm_head = tf.keras.layers.Dense(vocab_size)

    def call(self, idx, targets=None, training=False):
        B, T = idx.shape
        
        tok_emd = self.token_embedding_table(idx)  # (B, T, C)
        pos_emd = self.position_embedding_table(tf.range(T)) #(T, C)
        x = tok_emd + pos_emd #(B,T,C) # Holds Token identities & position
        x = self.sa_head(x) #Apply one head of self attention. (B,T,C) 
        logits = self.lm_head(x) # (B, T, vocab_size) # finally decoded here

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = tf.reshape(logits, (B*T, C))
            targets = tf.reshape(targets, (B*T,))
            loss = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets)
            )

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] #Because if idx is more than block size then our positional embd above
                                            #is gonna run out of scope because it has embeddings of upto block size
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]  # (B, C)
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            idx_next = tf.random.categorical(tf.math.log(probs), 1)  # (B, 1)
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx


In [175]:
model = BigramLanguageModel(vocab_size, n_embd)


logits, loss = model(xbatch, ybatch)
print(logits.shape)
print(loss)

(32, 65)
tf.Tensor(4.169185, shape=(), dtype=float32)


In [176]:
print(''.join(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int64), max_new_tokens=100).numpy()[0].tolist())))

#Total Garbage


ZiQlH
XYbrOU&py,dn-ovbsESOAqBKsk:yxV'-lNES!OU$3?xB,CpHqR

i;TFC,MjPQ3FbQ,SjCJ.hetJ.Mv!WQ?q,&A.M KHBO


In [178]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

batch_size = 32
num_steps = 10000

for step in range(num_steps):
    # Sample a batch of data
    xbatch, ybatch = get_batch('train')  # Assuming you have a function get_batch

    # Evaluate the loss
    with tf.GradientTape() as tape:
        logits, loss = model(xbatch, ybatch)

    # Compute gradients and update weights
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print('Loss = ', loss.numpy())


Loss =  2.4673042


In [179]:
#Somewhat stuctured results

print(''.join(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int64), max_new_tokens=500).numpy()[0].tolist())))


A or INGLOrle bere inmard meande LAnghall ysatan
Tousave foistha avende INGllit the wil acomear
Feder re yoom rne lem! whe my I hith tis DUCo ixpeparig:

CHour beetay pathe Anen. mak
HAs an P qu ben y ghe,
LENorolt fthad n I eathe thet Th g:
A'sourssid h t IXEY:
I:

ARYorve oll?
S:
Wisef s outleveryf Whond, d asive ntss s llamuine,
T: ifokes besuce d hatorul n suces he se rt lin he aumut?

I ok? s ee.
ARee y V:
I t ware f aves.

D:
Serssor'd undpirthouse ger lve lyore.
He tld Maldig ore sth Sean


In [ ]:
f